### 서울교통공사 지하철역 시간대별 지나간 지하철 대수 전처리
- https://data.seoul.go.kr/dataList/OA-101/A/1/datasetView.do

In [1]:
import requests

# 실제 서비스 키
SERVICE_KEY = '6c41706a496e617931394343774c44'
# 전체 지하철 역 코드를 저장할 리스트 초기화
subway_codes = []

# 전체 지하철 역 코드 목록을 가져오기 위한 URL
station_list_url = f'http://openAPI.seoul.go.kr:8088/{SERVICE_KEY}/json/SearchInfoBySubwayNameService/1/2/'

# API를 사용하여 전체 지하철 역 코드 목록을 가져오기
req = requests.get(station_list_url)
print(req)
print(req.text)

<Response [200]>
{"SearchInfoBySubwayNameService":{"list_total_count":777,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다"},"row":[{"STATION_CD":"0244","STATION_NM":"용답","LINE_NUM":"02호선","FR_CODE":"211-1"},{"STATION_CD":"0245","STATION_NM":"신답","LINE_NUM":"02호선","FR_CODE":"211-2"}]}}


In [2]:
if req.status_code == 200:
    data = req.json()
    if 'SearchInfoBySubwayNameService' in data:
        for item in data['SearchInfoBySubwayNameService']['row']:
            subway_code = item['STATION_CD']  # 역 코드 가져오기
            subway_codes.append(subway_code)
    else:
        print("데이터에 'SearchInfoBySubwayNameService'가 없습니다.")
else:
    print("역 코드 목록을 가져오는 데 실패했습니다.")

In [3]:
# 각 지하철 역 코드에 대한 데이터 수집
all_data = []  # 모든 역의 데이터를 저장할 리스트 초기화

# 모든 요일과 상하행에 대한 조합에 대해 데이터 수집
for subway_code in subway_codes:
    for week_tag in ['1', '2', '3']:  # 모든 요일에 대한 조합
        for inout_tag in ['1', '2']:  # 상하행에 대한 조합
            url = f'http://openAPI.seoul.go.kr:8088/{SERVICE_KEY}/json/SearchSTNTimeTableByIDService/1/2/{subway_code}/{week_tag}/{inout_tag}/'
            req = requests.get(url)

            if req.status_code == 200:
                data = req.json()
                all_data.append(data)  # 각 역의 데이터를 all_data 리스트에 추가
            else:
                print(f"역 코드 {subway_code}, 요일 {week_tag}, 상하행 {inout_tag}에 대한 데이터를 가져오는 데 실패했습니다.")

# all_data 리스트에는 각 역의 데이터가 저장되어 있습니다.

In [4]:
print(subway_codes)

['0244', '0245']


In [5]:
import pandas as pd

# all_data 리스트에 있는 모든 데이터를 하나의 데이터프레임으로 합치기
all_data_frames = []
for data in all_data:
    if 'SearchSTNTimeTableByIDService' in data:
        df = pd.DataFrame(data['SearchSTNTimeTableByIDService']['row'])
        all_data_frames.append(df)

# 모든 데이터프레임을 하나로 합치기
all_df = pd.concat(all_data_frames, ignore_index=True)


In [6]:
all_df.head(50)

,LINE_NUM,FR_CODE,STATION_CD,STATION_NM,TRAIN_NO,ARRIVETIME,LEFTTIME,ORIGINSTATION,DESTSTATION,SUBWAYSNAME,SUBWAYENAME,WEEK_TAG,INOUT_TAG,FL_FLAG,DESTSTATION2,EXPRESS_YN,BRANCH_LINE
0,02호선,211-1,0244,용답,1502,05:35:30,05:36:00,0246,0211,신설동,성수,1,1,,,G,
1,02호선,211-1,0244,용답,1504,05:52:30,05:53:00,0246,0211,신설동,성수,1,1,,,G,
2,02호선,211-1,0244,용답,1501,05:33:00,05:33:30,0211,0246,성수,신설동,1,2,,,G,
3,02호선,211-1,0244,용답,1503,05:46:00,05:46:30,0211,0246,성수,신설동,1,2,,,G,
4,02호선,211-1,0244,용답,1502,05:35:30,05:36:00,0246,0211,신설동,성수,2,1,,,G,
5,02호선,211-1,0244,용답,1504,05:50:30,05:51:00,0246,0211,신설동,성수,2,1,,,G,
6,02호선,211-1,0244,용답,1501,05:31:00,05:31:30,0211,0246,성수,신설동,2,2,,,G,
7,02호선,211-1,0244,용답,1503,05:45:00,05:45:30,0211,0246,성수,신설동,2,2,,,G,
8,02호선,211-1,0244,용답,1502,05:35:30,05:36:00,0246,0211,신설동,성수,3,1,,,G,
9,02호선,211-1,0244,용답,1504,05:50:30,05:51:00,0246,0211,신설동,성수,3,1,,,G,


In [9]:
all_df.LINE_NUM.value_counts()

02호선    24
Name: LINE_NUM, dtype: int64

In [10]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   LINE_NUM       24 non-null     object
 1   FR_CODE        24 non-null     object
 2   STATION_CD     24 non-null     object
 3   STATION_NM     24 non-null     object
 4   TRAIN_NO       24 non-null     object
 5   ARRIVETIME     24 non-null     object
 6   LEFTTIME       24 non-null     object
 7   ORIGINSTATION  24 non-null     object
 8   DESTSTATION    24 non-null     object
 9   SUBWAYSNAME    24 non-null     object
 10  SUBWAYENAME    24 non-null     object
 11  WEEK_TAG       24 non-null     object
 12  INOUT_TAG      24 non-null     object
 13  FL_FLAG        24 non-null     object
 14  DESTSTATION2   24 non-null     object
 15  EXPRESS_YN     24 non-null     object
 16  BRANCH_LINE    24 non-null     object
dtypes: object(17)
memory usage: 3.3+ KB


In [11]:
# 요일과 상하행 숫자를 글자로 매핑하는 딕셔너리 생성
week_mapping = {
    '1': '평일',
    '2': '토요일',
    '3': '휴일/일요일'
}

inout_mapping = {
    '1': '상행',
    '2': '하행'
}

In [12]:
unique_stations = all_df['STATION_CD'].unique()
unique_lines = all_df['LINE_NUM'].unique()
unique_week_tags = all_df['WEEK_TAG'].unique()
unique_inout_tags = all_df['INOUT_TAG'].unique()

# 필요한 시간대 컬럼 생성
time_columns = [f'{i:02d}시~{i+1:02d}시' for i in range(5, 25)]

# 결과 데이터프레임 초기화
result_df = pd.DataFrame(columns=['STATION_CD', 'STATION_NM', 'LINE_NUM', '요일', '상하행'] + time_columns)

# 각 역별로 데이터 처리
for station_cd in unique_stations:
    station_name = all_df[all_df['STATION_CD'] == station_cd]['STATION_NM'].iloc[0]
    for line_num in unique_lines:
        for week_tag in unique_week_tags:
            for inout_tag in unique_inout_tags:
                filtered_df = all_df[(all_df['STATION_CD'] == station_cd) &
                                     (all_df['LINE_NUM'] == line_num) &
                                     (all_df['WEEK_TAG'] == week_tag) &
                                     (all_df['INOUT_TAG'] == inout_tag)]

                # 시간대별 열차 수를 저장할 리스트 초기화
                train_counts = [0] * len(time_columns)

                for index, row in filtered_df.iterrows():
                    # hh:mm:ss 중 h만 가져오기
                    arr_time = int(row['ARRIVETIME'].split(':')[0])
                    # 첫 차: 5시 -> 0 index 부터 train_counts 에 저장
                    if 5 <= arr_time < 25:
                        train_counts[arr_time - 5] += 1

                # 모든 시간대가 0인 행은 추가하지 않음
                if all(count == 0 for count in train_counts):
                    continue

                # 결과 데이터프레임에 행 추가
                temp_df = pd.DataFrame({
                    'STATION_CD': [station_cd],
                    'STATION_NM': [station_name],
                    'LINE_NUM': [line_num],
                    '요일': [week_mapping[str(week_tag)]],
                    '상하행': [inout_mapping[str(inout_tag)]],
                    **{time_col: count for time_col, count in zip(time_columns, train_counts)}})

                result_df = pd.concat([result_df, temp_df], ignore_index=True)

In [13]:
# 결과 출력
result_df

,STATION_CD,STATION_NM,LINE_NUM,요일,상하행,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,...,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~25시
0,0244,용답,02호선,평일,상행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0244,용답,02호선,평일,하행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0244,용답,02호선,토요일,상행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0244,용답,02호선,토요일,하행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0244,용답,02호선,휴일/일요일,상행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0244,용답,02호선,휴일/일요일,하행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0245,신답,02호선,평일,상행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0245,신답,02호선,평일,하행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0245,신답,02호선,토요일,상행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0245,신답,02호선,토요일,하행,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   STATION_CD  12 non-null     object
 1   STATION_NM  12 non-null     object
 2   LINE_NUM    12 non-null     object
 3   요일          12 non-null     object
 4   상하행         12 non-null     object
 5   05시~06시     12 non-null     object
 6   06시~07시     12 non-null     object
 7   07시~08시     12 non-null     object
 8   08시~09시     12 non-null     object
 9   09시~10시     12 non-null     object
 10  10시~11시     12 non-null     object
 11  11시~12시     12 non-null     object
 12  12시~13시     12 non-null     object
 13  13시~14시     12 non-null     object
 14  14시~15시     12 non-null     object
 15  15시~16시     12 non-null     object
 16  16시~17시     12 non-null     object
 17  17시~18시     12 non-null     object
 18  18시~19시     12 non-null     object
 19  19시~20시     12 non-null     object
 20  20시~21시     

In [15]:
result_df[time_columns].value_counts()

05시~06시  06시~07시  07시~08시  08시~09시  09시~10시  10시~11시  11시~12시  12시~13시  13시~14시  14시~15시  15시~16시  16시~17시  17시~18시  18시~19시  19시~20시  20시~21시  21시~22시  22시~23시  23시~24시  24시~25시
2        0        0        0        0        0        0        0        0        0        0        0        0        0        0        0        0        0        0        0          12
dtype: int64

In [16]:
# 기초 통계량 확인
stats = result_df[time_columns].describe()
stats

,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,10시~11시,11시~12시,12시~13시,13시~14시,14시~15시,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~25시
count,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12
unique,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
top,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
freq,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12


In [18]:
'''
import os

# 분할된 데이터를 저장할 디렉토리 경로 설정
output_dir = '호선별_데이터_최종/'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# LINE_NUM 값에 따라 데이터 분할 및 저장
line_nums = ['01호선', '02호선', '03호선', '04호선', '05호선', '06호선', '07호선', '08호선']

for line_num in line_nums:
    # 조건에 맞는 데이터 추출
    subset_df = final_result_df[new_result_df['LINE_NUM'] == line_num]

    # CSV 파일로 저장 (파일명은 호선별로 구분)
    file_name = f'{line_num}.csv'
    subset_df.to_csv(output_dir + file_name, index=False)

print("데이터 분할 및 저장 완료.")
'''

'\nimport os\n\n# 분할된 데이터를 저장할 디렉토리 경로 설정\noutput_dir = \'호선별_데이터_최종/\'\n\n# 디렉토리가 존재하지 않으면 생성\nif not os.path.exists(output_dir):\n    os.makedirs(output_dir)\n\n# LINE_NUM 값에 따라 데이터 분할 및 저장\nline_nums = [\'01호선\', \'02호선\', \'03호선\', \'04호선\', \'05호선\', \'06호선\', \'07호선\', \'08호선\']\n\nfor line_num in line_nums:\n    # 조건에 맞는 데이터 추출\n    subset_df = final_result_df[new_result_df[\'LINE_NUM\'] == line_num]\n\n    # CSV 파일로 저장 (파일명은 호선별로 구분)\n    file_name = f\'{line_num}.csv\'\n    subset_df.to_csv(output_dir + file_name, index=False)\n\nprint("데이터 분할 및 저장 완료.")\n'